In [5]:
!pip install xlrd==2.0.1
!pip install pandas==1.1.5
!pip install numpy==1.19.5
!pip install scipy==1.1.0
!pip install openpyxl==3.0.9
!pip install pyproj==3.0.1
!pip install geopandas==0.8.0
!pip install matplotlib==3.3.4
!pip install folium==0.12.1.post1




In [6]:
!pip list

Package             Version
------------------- -------------------
alembic             0.9.5
asn1crypto          0.22.0
attrs               21.4.0
beautifulsoup4      4.5.3
bleach              2.0.0
bokeh               0.12.9
branca              0.5.0
certifi             2017.7.27.1
cffi                1.10.0
chardet             3.0.4
click               6.7
click-plugins       1.1.1
cligj               0.7.2
cloudpickle         0.2.2
conda               4.3.23
cryptography        1.9
cycler              0.10.0
Cython              0.25.2
dask                0.15.3
decorator           4.1.2
dill                0.2.6
distributed         1.19.1
docopt              0.6.2
entrypoints         0.2.3
et-xmlfile          1.1.0
fastcache           1.0.2
Fiona               1.8.21
folium              0.12.1.post1
geopandas           0.8.0
h5py                2.6.0
heapdict            1.0.0
html5lib            0.999999999
idna                2.5
ipykernel           4.6.1
ipython             6.2.1

In [7]:
import xlrd
import pandas as pd
import numpy as np
import scipy
import openpyxl

!pip install pyproj
from pyproj import Proj, transform

import warnings
warnings.filterwarnings('ignore')

import geopandas

# 1) Gemeindedaten einlesen

In [8]:
#URL für Datenbezug -> https://www.bfs.admin.ch/bfs/de/home/statistiken/kataloge-datenbanken/tabellen.assetdetail.15864450.html
#Dokument als xlsx gespeichert

excel_file = pd.ExcelFile('./Rohdaten/gemeinden 2021.xlsx', engine='openpyxl')
gemeinden_2021 = pd.read_excel(excel_file)

In [9]:
gemeinden_2021.head(10)

,Regionalporträts 2021: Gemeinden - Kennzahlen,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,T21.3.1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Bevölkerung,NaN,NaN,NaN,Altersverteilung in %,NaN,NaN,Bevölkerungsbewegung (in ‰),...,Wähleranteile ausgewählter Parteien in % (Nati...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gemeindecode,Gemeindename,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,FDP 2),CVP,SP,SVP,EVP/CSP,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien
5,NaN,NaN,2019,2010-2019,2019,2019,2019,2019,2019,2019,...,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,Schweiz,8606033,9.4,215.2,25.3,20,61.4,18.7,4.5,...,15.1,11.4,16.8,25.6,2.3,7.8,2.4,1,13.2,2.1
8,1,Aeugst am Albis,1981,8.60746,250.442,14.1848,19.788,62.1908,18.0212,7.06535,...,16.421,2.49797,11.5651,30.381,3.22747,14.3173,1.26372,0.16211,15.0505,3.82065
9,2,Affoltern am Albis,12303,10.9278,1161.76,28.7003,20.1658,61.6679,18.1663,5.54378,...,13.3854,4.53534,14.6453,30.3069,6.61856,12.6666,2.00145,0.198718,12.5906,1.55552


## 1.1) Löschen der NaN Zeilen

In [10]:
#Werte aus Zeile 4 als Spaltenüberschrift einsetzen
new_header = gemeinden_2021.iloc[4]
gemeinden_2021.columns = new_header

In [11]:
#Entfernen der leeren Zeilen vor der Tabelle und am Tabellenende
gemeinden_2021 = gemeinden_2021.drop(labels=range(0,7), axis=0)
gemeinden_2021 = gemeinden_2021.drop(labels=range(2180,2196), axis=0)

In [12]:
gemeinden_2021.head()

4,Gemeindecode,Gemeindename,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,FDP 2),CVP,SP,SVP,EVP/CSP,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien
7,NaN,Schweiz,8606033,9.4,215.2,25.3,20,61.4,18.7,4.5,...,15.1,11.4,16.8,25.6,2.3,7.8,2.4,1,13.2,2.1
8,1,Aeugst am Albis,1981,8.60746,250.442,14.1848,19.788,62.1908,18.0212,7.06535,...,16.421,2.49797,11.5651,30.381,3.22747,14.3173,1.26372,0.16211,15.0505,3.82065
9,2,Affoltern am Albis,12303,10.9278,1161.76,28.7003,20.1658,61.6679,18.1663,5.54378,...,13.3854,4.53534,14.6453,30.3069,6.61856,12.6666,2.00145,0.198718,12.5906,1.55552
10,3,Bonstetten,5572,8.00543,749.933,16.565,23.636,59.8349,16.5291,4.31655,...,15.355,3.86796,16.7429,26.1163,4.19402,16.2379,1.28403,0.176013,12.3685,1.38502
11,4,Hausen am Albis,3751,11.6369,275.809,16.0224,22.1808,59.824,17.9952,3.22061,...,11.1352,3.93455,13.7115,33.1848,4.05976,12.417,1.61075,0.19312,16.7441,1.61075


## 1.2) Nulls identifizieren

In [13]:
#Check dass keine Nulls im Dataframe
gemeinden_2021.isnull().sum().sum()

1

In [14]:
#1 NaN identifiziert. Da nur 1 NaN in der ganzen Tabelle wir der Wert CH als Füllwert eingesetzt da die Schweiz 
#keinen Gemeindecode besitzt.
gemeinden_2021 = gemeinden_2021.fillna('CH') #CH löschen

In [15]:
#Check dass keine Nulls im Dataframe
gemeinden_2021.isnull().sum().sum()

0

## 1.3) Import der Geodaten

In [16]:
#https://data.geo.admin.ch/ch.swisstopo-vd.ortschaftenverzeichnis_plz/PLZO_CSV_LV95.zip
#Start Import der Geodaten
geo = pd.read_csv('./Rohdaten/geodaten.csv', sep=';')
geo.head()

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,2679435.817,1235653.185,de
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,2679815.475,1237410.215,de
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,2675104.519,1237890.488,de
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,2676439.358,1236966.371,de
4,Bonstetten,8906,0,Bonstetten,3,ZH,2677798.571,1241024.843,de


In [17]:
#löschen der nicht benötigten columns
geo = geo.drop(['Ortschaftsname','PLZ','Gemeindename','Zusatzziffer','Sprache'], axis=1)
geo.shape

(4126, 4)

## 1.4) Geodaten transformieren von LV95 -> WGS84

In [18]:
#Convert LV95 to WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = geo['E'], geo['N']
x2,y2 = transform(inProj, outProj, x1,y1)

In [19]:
geo['WGS84_E'] = x2
geo['WGS84_N'] = y2

In [20]:
geo = geo.rename(columns = {'E':'LV95_E', 'N':'LV95_N', 'BFS-Nr':'Gemeindecode'})
geo.head()

,Gemeindecode,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
0,1,ZH,2679435.817,1235653.185,47.267004,8.488313
1,1,ZH,2679815.475,1237410.215,47.282761,8.493642
2,2,ZH,2675104.519,1237890.488,47.287633,8.431459
3,2,ZH,2676439.358,1236966.371,47.279169,8.448945
4,3,ZH,2677798.571,1241024.843,47.315510,8.467611


## 1.5) Merge der Geodaten und Gemeindedaten

In [21]:
#Kombinierte Geodaten und Gemeindedaten basierend auf dem Gemeindecode. Da die Geodaten alle Ortschaften pro PLZ enthält,
#werden in der Gemeindetabelle Duplikate generiert.
data_merged = gemeinden_2021.merge(geo, how='left', on=['Gemeindecode'])
data_merged.head()

,Gemeindecode,Gemeindename,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
0,CH,Schweiz,8606033,9.400000,215.200000,25.300000,20.000000,61.400000,18.700000,4.500000,...,7.8,2.4,1,13.2,2.1,NaN,NaN,NaN,NaN,NaN
1,1,Aeugst am Albis,1981,8.607456,250.442478,14.184755,19.787986,62.190813,18.021201,7.065355,...,14.3173,1.26372,0.16211,15.0505,3.82065,ZH,2679435.817,1235653.185,47.267004,8.488313
2,1,Aeugst am Albis,1981,8.607456,250.442478,14.184755,19.787986,62.190813,18.021201,7.065355,...,14.3173,1.26372,0.16211,15.0505,3.82065,ZH,2679815.475,1237410.215,47.282761,8.493642
3,2,Affoltern am Albis,12303,10.927779,1161.756374,28.700317,20.165813,61.667886,18.166301,5.543780,...,12.6666,2.00145,0.198718,12.5906,1.55552,ZH,2675104.519,1237890.488,47.287633,8.431459
4,2,Affoltern am Albis,12303,10.927779,1161.756374,28.700317,20.165813,61.667886,18.166301,5.543780,...,12.6666,2.00145,0.198718,12.5906,1.55552,ZH,2676439.358,1236966.371,47.279169,8.448945


In [22]:
data_merged.shape

(4114, 48)

In [23]:
#Die durch den merge generierten Duplikate werden wieder gelöscht. Es soll pro politische Gemeinde nur eine Ortschaft
#verwendet werden.
data_merged = data_merged.drop_duplicates(subset=['Gemeindename'], keep='first')

In [24]:
#Check ob Anzahl politische Gemeinden korrekt. Gemäss https://www.bfs.admin.ch/bfs/de/home/grundlagen/agvch.assetdetail.15144517.html
#Gibt es 2172 politische Gemeinden in der Schweiz inkl. Daten auf Stufe CH sollte das Dataframe 2173 Zeilen beinhalten.
data_merged.shape

(2173, 48)

## 1.5) Bereinigung fehlender und nicht vollständiger Daten

In [25]:
#Check of in einzelnen Columns Nulls vorhanden sind.
data_merged.isnull().sum()

Gemeindecode                                      0
Gemeindename                                      0
Einwohner                                         0
Veränderung in %                                  0
Bevölkerungs-dichte pro km²                       0
Ausländer in %                                    0
0-19 Jahre                                        0
20-64 Jahre                                       0
65 Jahre und mehr                                 0
Rohe Heiratssziffer                               0
Rohe Scheidungsziffer                             0
Rohe Geburtenziffer                               0
Rohe Sterbeziffer                                 0
Anzahl Privathaushalte                            0
Durchschnittliche Haushaltsgrösse in Personen     0
Gesamtfläche in km² 1)                            0
Siedlungsfläche in %                              0
Veränderung in ha                                 0
Landwirtschafts-fläche in %                       0
Veränderung 

In [26]:
#Nach dem Merge gibt es einzelne Gemeinden für welche keine Keine Geodaten zur Verfügung stehen. Es handelt sich dabei
#um sehr kleine Gemeinden die auch sonst kaum Daten zur Verfügung stellen. Dies werden entfernt.
data_merged = data_merged.dropna()
data_merged.shape

(2142, 48)

In [27]:
#Dataframe anhand der Einwohnerzahl sortieren "Aufsteigend"
data_merged = data_merged.sort_values(by='Einwohner', ascending=True)
data_merged.head()

,Gemeindecode,Gemeindename,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
1587,2549,Kammersrohr,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,10.1852,0.925926,*,7.40741,11.1111,SO,2611701.146,1233572.044,47.252957,7.593204
3754,6172,Bister,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,3.28947,*,*,10.5263,*,VS,2648289.945,1134692.620,46.361879,8.066106
633,708,Schelten,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,5.88235,0,0.490196,30.8824,0,BE,2608514.796,1242354.134,47.331996,7.551276
849,972,Berken,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,4.31548,7.29167,0,0.595238,0,BE,2620245.695,1230572.848,47.225775,7.705946
638,715,Rebévelier,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,0,0.320513,3.52564,8.49359,3.52564,BE,2581390.049,1237837.181,47.291158,7.192605


In [28]:
#Damit statistische Auswertungen möglich sind, müssen sämtliche Columns bis auf Gemeindename und Kürzel nur Numerische Werte
#Werte enthalten. Dies wird wie folgt überprüft
data_merged.applymap(np.isreal).sum()

Gemeindecode                                     2142
Gemeindename                                        0
Einwohner                                        2142
Veränderung in %                                 2142
Bevölkerungs-dichte pro km²                      2142
Ausländer in %                                   2142
0-19 Jahre                                       2142
20-64 Jahre                                      2142
65 Jahre und mehr                                2142
Rohe Heiratssziffer                              2142
Rohe Scheidungsziffer                            2142
Rohe Geburtenziffer                              2142
Rohe Sterbeziffer                                2142
Anzahl Privathaushalte                           2142
Durchschnittliche Haushaltsgrösse in Personen    2142
Gesamtfläche in km² 1)                           2142
Siedlungsfläche in %                             2142
Veränderung in ha                                2142
Landwirtschafts-fläche in % 

In [29]:
#Die übrigen nicht numerischen Werte der Statistischen Daten wurden im Datensatz mit einem * oder X gefüllt.
#Diese Werte werden mit der ffill Methode überschrieben. Da der Datensatz zuvor anhand der Einwohnerzahl geordnet wurde,
#werden deshalb bei Fehldaten die Werte der nächst kleineren Gemeinde kopiert. 
data_merged = data_merged.replace('*', np.nan).ffill()
data_merged = data_merged.replace('X', np.nan).ffill()

In [30]:
#Check Nulls
data_merged.isnull().sum().sum()

48

In [31]:
#Für einzelne Columns fehlte auf der 1. Zeile ein Wert, weshalb durch ffill nicht alle Daten gefüllt werden konnten.
#Dies wird durch bfill korrigiert.
data_merged = data_merged.fillna(method = 'bfill')
data_merged.head()

,Gemeindecode,Gemeindename,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
1587,2549,Kammersrohr,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,10.185185,0.925926,0.490196,7.407407,11.111111,SO,2611701.146,1233572.044,47.252957,7.593204
3754,6172,Bister,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,3.289474,0.925926,0.490196,10.526316,11.111111,VS,2648289.945,1134692.620,46.361879,8.066106
633,708,Schelten,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,5.882353,0.000000,0.490196,30.882353,0.000000,BE,2608514.796,1242354.134,47.331996,7.551276
849,972,Berken,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,4.315476,7.291667,0.000000,0.595238,0.000000,BE,2620245.695,1230572.848,47.225775,7.705946
638,715,Rebévelier,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,0.000000,0.320513,3.525641,8.493590,3.525641,BE,2581390.049,1237837.181,47.291158,7.192605


In [32]:
#Check Nulls
data_merged.isnull().sum().sum()

0

In [33]:
#Check, dass bis auf Gemeindename und Kantonskürzel alle columns numerische Werte ausweisen.
data_merged.applymap(np.isreal).sum()

Gemeindecode                                     2142
Gemeindename                                        0
Einwohner                                        2142
Veränderung in %                                 2142
Bevölkerungs-dichte pro km²                      2142
Ausländer in %                                   2142
0-19 Jahre                                       2142
20-64 Jahre                                      2142
65 Jahre und mehr                                2142
Rohe Heiratssziffer                              2142
Rohe Scheidungsziffer                            2142
Rohe Geburtenziffer                              2142
Rohe Sterbeziffer                                2142
Anzahl Privathaushalte                           2142
Durchschnittliche Haushaltsgrösse in Personen    2142
Gesamtfläche in km² 1)                           2142
Siedlungsfläche in %                             2142
Veränderung in ha                                2142
Landwirtschafts-fläche in % 

In [34]:
#Gemeindename als Index einsetzen
data_merged = data_merged.set_index('Gemeindename')
data_merged.head()

,Gemeindecode,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,Rohe Scheidungsziffer,...,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
Gemeindename,,,,,,,,,,,,,,,,,,,,,
Kammersrohr,2549,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,0.0,...,10.185185,0.925926,0.490196,7.407407,11.111111,SO,2611701.146,1233572.044,47.252957,7.593204
Bister,6172,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,0.0,...,3.289474,0.925926,0.490196,10.526316,11.111111,VS,2648289.945,1134692.620,46.361879,8.066106
Schelten,708,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,0.0,...,5.882353,0.000000,0.490196,30.882353,0.000000,BE,2608514.796,1242354.134,47.331996,7.551276
Berken,972,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,0.0,...,4.315476,7.291667,0.000000,0.595238,0.000000,BE,2620245.695,1230572.848,47.225775,7.705946
Rebévelier,715,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,0.0,...,0.000000,0.320513,3.525641,8.493590,3.525641,BE,2581390.049,1237837.181,47.291158,7.192605


In [35]:
#Zwischenspeichern der Daten
data_merged.to_csv('./Daten/Gemeinde inkl. geo.csv')
data_merged.to_excel('./Daten/Gemeinde inkl. geo.xlsx')

# 1.6) Verarbeitung des Shapefiles für Choropleths

In [36]:
#Verarbeiten der Kantonsgrenzen der Schweiz

#URL der Daten: https://www.swisstopo.admin.ch/de/geodata/landscape/boundaries3d.html

#https://stackoverflow.com/questions/43119040/shapefile-into-geojson-conversion-python-3
shp_file = geopandas.read_file('./Rohdaten/kanton.shp')

In [37]:
#Umwandlung LV95 -> WGS84
shp_file= shp_file.set_crs(2056, allow_override=True)
shp_file= shp_file.to_crs(4326)

In [38]:
#Speichern des JSON files
shp_file.to_file('./Daten/Kantonsgrenzen.geojson', driver='GeoJSON')

# 2) Anreichern der Daten

In [39]:
#Einlesen des Zwischengespeicherten Datensatzes
df_gemeinden = pd.read_csv('./Daten/Gemeinde inkl. geo.csv')

In [40]:
df_gemeinden.head()

,Gemeindename,Gemeindecode,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,GLP,BDP,PdA/Sol.,GPS,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N
0,Kammersrohr,2549,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,10.185185,0.925926,0.490196,7.407407,11.111111,SO,2611701.146,1233572.044,47.252957,7.593204
1,Bister,6172,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,3.289474,0.925926,0.490196,10.526316,11.111111,VS,2648289.945,1134692.620,46.361879,8.066106
2,Schelten,708,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,5.882353,0.000000,0.490196,30.882353,0.000000,BE,2608514.796,1242354.134,47.331996,7.551276
3,Berken,972,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,4.315476,7.291667,0.000000,0.595238,0.000000,BE,2620245.695,1230572.848,47.225775,7.705946
4,Rebévelier,715,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,0.000000,0.320513,3.525641,8.493590,3.525641,BE,2581390.049,1237837.181,47.291158,7.192605


## 2.1) Politische Kategorisierung einfügen

In [41]:
#Einfügen der Politischen Kategorisierung nach dem Schema Links - Mitte - Rechts.
#Grundlage gemäss Visualisierung: https://ch-info.swiss/edition-2020/direkte-demokratie/parteien-von-links-nach-rechts
df_gemeinden['Linksparteien'] = df_gemeinden['SP']+df_gemeinden['PdA/Sol.']+df_gemeinden['GPS']
df_gemeinden['Mitteparteien'] = df_gemeinden['CVP']+df_gemeinden['EVP/CSP']+df_gemeinden['GLP']+df_gemeinden['BDP']
df_gemeinden['Rechtsparteien'] = df_gemeinden['FDP 2)']+df_gemeinden['SVP']+df_gemeinden['Kleine Rechtsparteien']

## 2.2) Ersetzen Kantonskürzel durch Kantonsname

In [42]:
kanton_lookup = {
    'AG':'Aargau',
    'AI':'Appenzell Innerrhoden',
    'AR':'Appenzell Ausserrhoden',
    'BE':'Bern',
    'BL':'Basel-Landschaft',
    'BS':'Basel-Stadt',
    'FR':'Fribourg',
    'GE':'Genève',
    'GL':'Glarus',
    'GR':'Graubünden',
    'JU':'Jura',
    'LU':'Luzern',
    'NE':'Neuchâtel',
    'NW':'Nidwalden',
    'OW':'Obwalden',
    'SG':'St. Gallen',
    'SH':'Schaffhausen',
    'SO':'Solothurn',
    'SZ':'Schwyz',
    'TG':'Thurgau',
    'TI':'Ticino',
    'UR':'Uri',
    'VD':'Vaud',
    'VS':'Valais',
    'ZG':'Zug',
    'ZH':'Zürich',
}

In [43]:
type(kanton_lookup)

dict

In [44]:
#Duplikat des Kantontskürzels erstellen
df_gemeinden['Kantonskürzel2'] = df_gemeinden['Kantonskürzel']
#Katnonskürzel2 wird durch Kantonsname ersetzt. Wird später bei Choropleths benötigt
df_gemeinden = df_gemeinden.replace({'Kantonskürzel2':kanton_lookup})
df_gemeinden = df_gemeinden.rename(columns = {'Kantonskürzel2':'Kantonsname'})
df_gemeinden.head()

,Gemeindename,Gemeindecode,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,Kleine Rechtsparteien,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N,Linksparteien,Mitteparteien,Rechtsparteien,Kantonsname
0,Kammersrohr,2549,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,11.111111,SO,2611701.146,1233572.044,47.252957,7.593204,18.082789,35.185185,45.370370,Solothurn
1,Bister,6172,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,11.111111,VS,2648289.945,1134692.620,46.361879,8.066106,25.490196,48.952242,38.084795,Valais
2,Schelten,708,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,0.000000,BE,2608514.796,1242354.134,47.331996,7.551276,37.990196,17.647059,35.784314,Bern
3,Berken,972,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,0.000000,BE,2620245.695,1230572.848,47.225775,7.705946,4.464286,11.607143,80.803571,Bern
4,Rebévelier,715,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,3.525641,BE,2581390.049,1237837.181,47.291158,7.192605,17.467949,15.384615,50.801282,Bern


In [45]:
#Erstellen eines Dictionary der Regionen pro Kanton
region_dict =	{
  "Ostschweiz": ["SG", "TG", "SH", "AR", "AI", "GL", "GR"],
  "Genferseeregion": ["GE", "VD", "VS"],
  "EspaceMittelland": ["BE", "FR", "SO", "NE", "JU"],
  "Nordwestschweiz": ["BL", "BS", "AG"],
  "Zürich" : ["ZH"],
  "Zentralschweiz" : ["LU", "UR", "SZ", "NW", "OW", "ZG"],
  "Tessin": ["TI"]
}

In [46]:
#Key Value change des Dict region_dict
kanton_region= dict()
for region in region_dict.items():
    for kantone in region[1:]:
        for kanton in kantone:
            kanton_region[kanton] = region[0]


kanton_region

{'AG': 'Nordwestschweiz',
 'AI': 'Ostschweiz',
 'AR': 'Ostschweiz',
 'BE': 'EspaceMittelland',
 'BL': 'Nordwestschweiz',
 'BS': 'Nordwestschweiz',
 'FR': 'EspaceMittelland',
 'GE': 'Genferseeregion',
 'GL': 'Ostschweiz',
 'GR': 'Ostschweiz',
 'JU': 'EspaceMittelland',
 'LU': 'Zentralschweiz',
 'NE': 'EspaceMittelland',
 'NW': 'Zentralschweiz',
 'OW': 'Zentralschweiz',
 'SG': 'Ostschweiz',
 'SH': 'Ostschweiz',
 'SO': 'EspaceMittelland',
 'SZ': 'Zentralschweiz',
 'TG': 'Ostschweiz',
 'TI': 'Tessin',
 'UR': 'Zentralschweiz',
 'VD': 'Genferseeregion',
 'VS': 'Genferseeregion',
 'ZG': 'Zentralschweiz',
 'ZH': 'Zürich'}

In [47]:
#Erstellen der Column Region durch die definierte Region
df_gemeinden['Region'] = df_gemeinden['Kantonskürzel']
df_gemeinden = df_gemeinden.replace({'Region':kanton_region})
df_gemeinden = df_gemeinden.rename(columns = {'Region':'Region'})
df_gemeinden.head()

,Gemeindename,Gemeindecode,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,Kantonskürzel,LV95_E,LV95_N,WGS84_E,WGS84_N,Linksparteien,Mitteparteien,Rechtsparteien,Kantonsname,Region
0,Kammersrohr,2549,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,SO,2611701.146,1233572.044,47.252957,7.593204,18.082789,35.185185,45.370370,Solothurn,EspaceMittelland
1,Bister,6172,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,VS,2648289.945,1134692.620,46.361879,8.066106,25.490196,48.952242,38.084795,Valais,Genferseeregion
2,Schelten,708,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,BE,2608514.796,1242354.134,47.331996,7.551276,37.990196,17.647059,35.784314,Bern,EspaceMittelland
3,Berken,972,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,BE,2620245.695,1230572.848,47.225775,7.705946,4.464286,11.607143,80.803571,Bern,EspaceMittelland
4,Rebévelier,715,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,BE,2581390.049,1237837.181,47.291158,7.192605,17.467949,15.384615,50.801282,Bern,EspaceMittelland


Der Altersquotient ist eine wichtige demographische Kennziffer, bei der die Anzahl der älteren Menschen zur Anzahl der erwerbsfähigen Menschen gesetzt wird. Zur Berechnung können verschiedene Altersgrenzen gezogen werden. Beispielsweise können alle Einwohner über 65 Jahre als Ältere und Personen zwischen 15 und 64 Jahren als erwerbsfähig eingestuft werden. Mit dem Altersquotient können Rückschlüsse auf das Rentner-Erwerbsverhältnis gezogen werden. Dem Altersquotient steht der Jugendquotient gegenüber, bei dem die Anzahl der Kinder und Jugendlichen zur Anzahl der Erwerbsfähigen ins Verhältnis gesetzt wird. Werden Altersquotient und Jugendquotient addiert, ergibt sich Gesamtlastquotient. Aus dieser Zahl können Belastungen für die Sozialsysteme abgeleitet werden. Für den Altersquotient gibt es auch die Begriffe Alterslastquote oder Englisch: aging quotient.

#Demografische Kennzahlen: https://de.wikipedia.org/wiki/Abh%C3%A4ngigenquotient
Aufgrund der vorliegende Zahlen wurden die Definition von Wikipedia angepasst - 


#Abhängigenquotient

Der Abhängigenquotient ist ein Begriff aus der Demografie. Er gibt das Verhältnis der Anzahl von Personen, die nicht im Erwerbstätigenalter sind, zur Anzahl von Personen im Erwerbstätigenalter in einer Gesellschaft an.

Der Abhängigenquotient ergibt sich aus zwei Komponenten. Der Altenquotient gibt das Verhältnis der Anzahl älterer Menschen zur Anzahl jüngerer Menschen in einer Gesellschaft an und wird üblicherweise genauer als das Verhältnis der Anzahl von Personen, die nicht mehr im Erwerbstätigenalter sind, zur Anzahl von Personen im Erwerbstätigenalter definiert. Weitgehend synonym zum Begriff „Altenquotient“ werden die Begriffe „Altersquotient“, „Belastungsquotient“ oder auch „Altenlastquote“ verwendet. Der Jugendquotient stellt die zweite Komponente dar und gibt entsprechend das Verhältnis der Anzahl von Personen, die noch nicht im Erwerbstätigenalter sind, zur Anzahl von Personen im Erwerbstätigenalter an.

Diese Quotienten beziehen sich ausschließlich auf die Altersgrenzen und lassen Fragen einer tatsächlichen Erwerbstätigkeit und Erwerbsfähigkeit der Personen außer Betracht.
Inhaltsverzeichnis

    1 Berechnung
    2 Aktuelle Daten
    3 Kritik
    4 Einzelnachweise

Berechnung

Der Altenquotient gibt das statistische Verhältnis der Menschen, die (meist ab 65 bzw. 60 Jahre) nicht mehr im erwerbsfähigen Alter sind, zu jenen im erwerbsfähigen Alter (meist ab 15 bzw. 20 Jahre bis 64 bzw. 59 Jahre) an. Er bezieht sich dabei immer auf eine bestimmte Bevölkerung zu einem bestimmten Zeitpunkt. Üblich ist die Berechnung eines Altenquotienten aus der Zahl von Personen im Alter ab 65 Jahren bezogen auf die Zahl der 15- bis 64-Jährigen (also bezogen auf die Zahl der Personen im typischen Erwerbsalter). Eine abweichende Wahl von Altersgrenzen ist jedoch gleichfalls möglich. Die verwendeten Grenzen sollten entsprechend immer kenntlich gemacht werden.

Der Jugendquotient gibt entsprechend üblicherweise die Zahl der unter 15-Jährigen bezogen auf die Zahl der 15- bis 64-Jährigen an.

Der Abhängigenquotient gibt die Zahl derjeniger, die unter 15 oder über 64 Jahre alt sind, wieder auf die Zahl der 15- bis 64-Jährigen bezogen. Er ergibt sich daher als Summe aus dem Alten- und dem Jugendquotient.

    Altenquotient = Bevölkerung ab 65 Jahre Bevölkerung 15–64 Jahre {\displaystyle {\text{Altenquotient}}={\frac {\text{Bevölkerung ab 65 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}} {\displaystyle {\text{Altenquotient}}={\frac {\text{Bevölkerung ab 65 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}}

    Jugendquotient = Bevölkerung 0–14 Jahre Bevölkerung 15–64 Jahre {\displaystyle {\text{Jugendquotient}}={\frac {\text{Bevölkerung 0–14 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}} {\displaystyle {\text{Jugendquotient}}={\frac {\text{Bevölkerung 0–14 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}}

    Abhängigenquotient = Bevölkerung 0–14 Jahre oder ab 65 Jahre Bevölkerung 15–64 Jahre = Altenquotient + Jugendquotient {\displaystyle {\text{Abhängigenquotient}}={\frac {\text{Bevölkerung 0–14 Jahre oder ab 65 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}={\text{Altenquotient}}+{\text{Jugendquotient}}} {\displaystyle {\text{Abhängigenquotient}}={\frac {\text{Bevölkerung 0–14 Jahre oder ab 65 Jahre}}{\text{Bevölkerung 15–64 Jahre}}}={\text{Altenquotient}}+{\text{Jugendquotient}}}

Der errechnete Quotient kann jeweils als Dezimalzahl mit Nachkommastellen angegeben werden (z. B. durch die Angabe eines Altenquotienten 0,20 bei einer Relation von 1 zu 5). Üblich ist eine Multiplikation des Wertes mit dem Faktor 100 (im genannten Beispiel würde entsprechend der Altenquotient 20 im Sinne von 20 Älteren je 100 Jüngere resultieren).

Bisweilen findet sich auch eine Angabe als Prozentwert. Eine Angabe als Prozentwert ist allerdings streng genommen verkehrt und sollte vermieden werden, da der Quotient immer eine Ratio bzw. ein Größenverhältnis von zwei unterschiedlichen Populationen beschreibt. Prozentwerte erfassen demgegenüber meistens einen Teil eines Ganzen. Da in der mathematischen Definition der Nenner nicht die Gesamtpopulation, sondern die Population im Erwerbstätigenalter eingeht, wäre eine Prozentangabe somit irreführend. Deutlich wird dies an einem hypothetischen Zahlenbeispiel: bei einem Abhängigenquotient von 1,0 (bzw. 100) gäbe es altersbezogen ebenso viele Personen innerhalb wie außerhalb des Erwerbstätigenalters; wäre er als „100 %“ angegeben, würde fälschlicherweise der Eindruck erweckt, das Alter sämtlicher Personen läge außerhalb des Erwerbstätigenalters. 

In [48]:
#Berechnung von Altenquotien, Jugendquotient und Abhängigenquotient
df_gemeinden['Altenquotient'] = df_gemeinden['65 Jahre und mehr']/df_gemeinden['20-64 Jahre']
df_gemeinden['Jugendquotient'] = df_gemeinden['0-19 Jahre']/df_gemeinden['20-64 Jahre']
df_gemeinden['Abhaengigenquotient'] = df_gemeinden['Altenquotient'] + df_gemeinden['Jugendquotient']
df_gemeinden.head()

,Gemeindename,Gemeindecode,Einwohner,Veränderung in %,Bevölkerungs-dichte pro km²,Ausländer in %,0-19 Jahre,20-64 Jahre,65 Jahre und mehr,Rohe Heiratssziffer,...,WGS84_E,WGS84_N,Linksparteien,Mitteparteien,Rechtsparteien,Kantonsname,Region,Altenquotient,Jugendquotient,Abhaengigenquotient
0,Kammersrohr,2549,32,-17.948718,33.684211,3.125000,12.500000,62.500000,25.000000,0.000000,...,47.252957,7.593204,18.082789,35.185185,45.370370,Solothurn,EspaceMittelland,0.400000,0.200000,0.600000
1,Bister,6172,33,0.000000,5.679862,6.060606,9.090909,60.606061,30.303030,0.000000,...,46.361879,8.066106,25.490196,48.952242,38.084795,Valais,Genferseeregion,0.500000,0.150000,0.650000
2,Schelten,708,35,-14.634146,6.294964,5.714286,17.142857,65.714286,17.142857,28.169014,...,47.331996,7.551276,37.990196,17.647059,35.784314,Bern,EspaceMittelland,0.260870,0.260870,0.521739
3,Berken,972,41,-16.326531,29.496403,2.439024,2.439024,65.853659,31.707317,0.000000,...,47.225775,7.705946,4.464286,11.607143,80.803571,Bern,EspaceMittelland,0.481481,0.037037,0.518519
4,Rebévelier,715,42,-10.638298,11.830986,2.380952,26.190476,47.619048,26.190476,24.096386,...,47.291158,7.192605,17.467949,15.384615,50.801282,Bern,EspaceMittelland,0.550000,0.550000,1.100000


In [49]:
#Einlesen des Files Raumgliederungen zum Mappen nach Stadt, Agglomeration, Land.
#URL: https://www.agvchapp.bfs.admin.ch/de/typologies/query
excel_file = pd.ExcelFile("./Rohdaten/raumgliederungen.xlsx", engine='openpyxl')
raumgliederungen = pd.read_excel(excel_file)

In [50]:
raumgliederungen.head()


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Agglomerationen 2012
0,BFS Gde-nummer,Gemeindename,Kantons-nummer,Kanton,Bezirks-nummer,Bezirksname,Agglomerationen 2012
1,NaN,NaN,NaN,NaN,NaN,NaN,CH1+CL_AGGL+2012.0
2,1,Aeugst am Albis,1,ZH,101,Affoltern,261
3,2,Affoltern am Albis,1,ZH,101,Affoltern,261
4,3,Bonstetten,1,ZH,101,Affoltern,261


In [51]:
#Column Header einsetzen
new_header = raumgliederungen.iloc[0]
raumgliederungen.columns = new_header
raumgliederungen = raumgliederungen.drop([0, 1],)

In [52]:
#Definieren der Funktion zur Zuweisung des Agglo-Status auf die einzelnen Gemeinden.
def f(row):
    if row['BFS Gde-nummer'] == row['Agglomerationen 2012']:
        val = 'Stadt'
    elif 0 < row['Agglomerationen 2012']:
        val = 'Agglomeration'
    else:
        val = 'Land'
    return val

In [53]:
print(raumgliederungen.dtypes)

# using dictionary to convert specific columns
convert_dict = {'BFS Gde-nummer': int,
                'Agglomerationen 2012': int
               }
  
raumgliederungen  = raumgliederungen.astype(convert_dict)
print(raumgliederungen.dtypes)

0
BFS Gde-nummer          object
Gemeindename            object
Kantons-nummer          object
Kanton                  object
Bezirks-nummer          object
Bezirksname             object
Agglomerationen 2012    object
dtype: object
0
BFS Gde-nummer           int64
Gemeindename            object
Kantons-nummer          object
Kanton                  object
Bezirks-nummer          object
Bezirksname             object
Agglomerationen 2012     int64
dtype: object


In [54]:
#Einfügen des Agglo-Status in einer neuen column
raumgliederungen['Charakter'] =  raumgliederungen.apply(f, axis=1)

In [55]:
#Entfernen nicht notwendiger columns
raumgliederungen = raumgliederungen.drop(['Gemeindename', 'Kantons-nummer', 'Kanton'], axis=1)

In [56]:
raumgliederungen.head()

,BFS Gde-nummer,Bezirks-nummer,Bezirksname,Agglomerationen 2012,Charakter
2,1,101,Affoltern,261,Agglomeration
3,2,101,Affoltern,261,Agglomeration
4,3,101,Affoltern,261,Agglomeration
5,4,101,Affoltern,261,Agglomeration
6,5,101,Affoltern,261,Agglomeration


In [57]:
#Zusammenführen des Raumgliederungs Datensatz mit dem Gemeinde Dataframe
df_gemeinden = pd.merge(df_gemeinden, raumgliederungen, left_on='Gemeindecode', right_on='BFS Gde-nummer', how='left').drop(['BFS Gde-nummer','Bezirks-nummer','Bezirksname'], axis=1)

In [58]:
df_gemeinden.shape

(2142, 58)

In [59]:
#Überprüfen ob neue Null's entstanden sind
df_gemeinden.isnull().sum()

Gemeindename                                     0
Gemeindecode                                     0
Einwohner                                        0
Veränderung in %                                 0
Bevölkerungs-dichte pro km²                      0
Ausländer in %                                   0
0-19 Jahre                                       0
20-64 Jahre                                      0
65 Jahre und mehr                                0
Rohe Heiratssziffer                              0
Rohe Scheidungsziffer                            0
Rohe Geburtenziffer                              0
Rohe Sterbeziffer                                0
Anzahl Privathaushalte                           0
Durchschnittliche Haushaltsgrösse in Personen    0
Gesamtfläche in km² 1)                           0
Siedlungsfläche in %                             0
Veränderung in ha                                0
Landwirtschafts-fläche in %                      0
Veränderung in ha.1            

In [60]:
#Da das Excelfile zur Raumgliederung von 2012 ist gibt es kleine Differenzen bei der Anzahl Gemeinden und 5 weisen keinen Aglo-Status aus.
#Die nicht definierbaren Gemeinden erhalten den Status Land.
df_gemeinden[['Agglomerationen 2012']] = df_gemeinden[['Agglomerationen 2012']].fillna(value=0.0)
df_gemeinden[['Charakter']] = df_gemeinden[['Charakter']].fillna(value='Land')

In [61]:
df_gemeinden.isnull().sum().sum()

0

## 2.3 Speichern finale Gemeindedaten

In [62]:
df_gemeinden = df_gemeinden.set_index('Gemeindename')

In [63]:
df_gemeinden.to_csv('./Daten/Gemeinden_Final.csv')